# Modelo ARIMAX con variables exógenas del Banco Mundial
Este notebook entrena un modelo ARIMAX usando tres variables exógenas descargadas desde la API del World Bank:
- Deuda externa (External debt stocks)
- Tasa de desempleo
- Tasa de interés de préstamos

In [ ]:
import pandas as pd
import numpy as np
import wbdata
import datetime
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

## Parte 1: Carga y preparación de datos

In [ ]:
# Fecha de inicio y país
start_date = datetime.datetime(2000, 1, 1)
end_date = datetime.datetime(2024, 12, 31)
country = 'ESP'  # España

# Indicadores del Banco Mundial
indicators = {
    'DT.DOD.DECT.CD': 'deuda_externa',
    'SL.UEM.TOTL.ZS': 'desempleo',
    'FR.INR.LEND': 'tasa_interes'
}

# Descargar datos del Banco Mundial
exog_df = wbdata.get_dataframe(indicators, country=country, data_date=(start_date, end_date))
exog_df = exog_df[::-1]  # ordenar por fecha ascendente
exog_df = exog_df.fillna(method='ffill').dropna()

# Simulación de una serie objetivo (puedes reemplazar por tu serie real)
fecha_rng = pd.date_range(start='2000-01', end='2024-12', freq='M')
np.random.seed(42)
inflacion = pd.Series(np.random.normal(loc=2, scale=0.5, size=len(fecha_rng)), index=fecha_rng)
inflacion.name = 'inflacion'

# Resample y merge
exog_df = exog_df.resample('M').mean()
data = pd.concat([inflacion, exog_df], axis=1).dropna()

## Parte 2: Escalado y separación de datos

In [ ]:
train_size = int(len(data) * 0.8)
train, test = data.iloc[:train_size], data.iloc[train_size:]

# Escalar las variables exógenas
scaler = StandardScaler()
exog_train = scaler.fit_transform(train[['deuda_externa', 'desempleo', 'tasa_interes']])
exog_test = scaler.transform(test[['deuda_externa', 'desempleo', 'tasa_interes']])

## Parte 3: Entrenamiento del modelo ARIMAX

In [ ]:
model = SARIMAX(train['inflacion'],
                exog=exog_train,
                order=(1, 0, 1),
                enforce_stationarity=False,
                enforce_invertibility=False)

results = model.fit(disp=False)

## Parte 4: Pronóstico

In [ ]:
forecast = results.get_forecast(steps=len(test), exog=exog_test)
predicted_mean = forecast.predicted_mean
conf_int = forecast.conf_int()

## Parte 5: Visualización del pronóstico

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(train.index, train['inflacion'], label='Entrenamiento')
plt.plot(test.index, test['inflacion'], label='Real')
plt.plot(test.index, predicted_mean, label='Pronóstico')
plt.fill_between(test.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.title('Pronóstico ARIMAX de la Inflación')
plt.legend()
plt.grid(True)
plt.show()

## Parte 6: Evaluación del modelo

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = np.sqrt(mean_squared_error(test['inflacion'], predicted_mean))
mae = mean_absolute_error(test['inflacion'], predicted_mean)
print(f"RMSE: {rmse:.3f}")
print(f"MAE: {mae:.3f}")